# Análisis de costos de cursos en el mercado

In [57]:
import pandas as pd
import numpy as np
import os

# Rutas de archivos

In [58]:
path = os.path.abspath(os.path.join(os.getcwd(), '..','2_data_understanding_module','collecting_initial_data','existing_data','DIRTY_DATA'))
path_additional_data = os.path.abspath(os.path.join(os.getcwd(), '..','2_data_understanding_module','collecting_initial_data','additional_data'))
path_dwh = os.path.abspath(os.path.join(os.getcwd(),'DATAWAREHOUSE'))
path_ppd = os.path.abspath(os.path.join(os.getcwd(),'CLEANED_DATA'))

In [3]:
# Cargar los archivos de costos de educacion continua, costos de cursos cec y cursos de OC, OEC y CI

df_dim_perfil = pd.read_csv(os.path.join(path_dwh,'datamart_oec','dim_perfil.csv'))
df_dim_curso = pd.read_csv(os.path.join(path_dwh,'datamart_ci','dim_curso.csv'))


### Palabras clave

In [4]:
keyword = ['Python', 'programación web', 'paginas web', 'datos', 'Excel', 'Word', 'power point', 'BI', 'inteligencia de negocios', 'powerBI', 
'big data', 'programación', 'marketing', 'web', 'html', 'css',  'java', 'javascript', 'base de datos', 'redes', 'telecomunicaciones', 'Android', 'IOS', 'ARDUINO'
'Business Intelligence', 
'Desarrollo de software',
'Fibra Óptica',
'Gerencia de sistemas',
'Hackeo Ético',
'Informática Forense',
'Linux', 'WINDOWS'
'Office',
'PHP',
'Project',
'software R', 'Tableu', 'rapidminer', 
'Mysql', 'sql', 'sqlserver', 'Oracle', 'postgresql',
'Seguridad informática',
'Aulas virtuales', 'Moodle', 'software',
'Wordpress', 'joomla', 'xoops', 'drupal'
]

keyword = np.char.lower(np.array(keyword))

In [59]:
from hermetrics.levenshtein import Levenshtein
lev = Levenshtein()

In [12]:
#Filtrar las areas de Tecnologia y TICs

df_dim_curso_perfil = pd.read_csv(os.path.join(path_dwh,'datamart_oc','dim_curso_perfil.csv'))
df_dim_curso_perfil = df_dim_curso_perfil[df_dim_curso_perfil['area_familia'] == 'TECNOLOGÍAS DE LA INFORMACIÓN Y COMUNICACIÓN']

In [60]:
df_coec = pd.read_csv(os.path.join(path_ppd,'costos_cursos_educacion_continua.csv'))


Index(['Nombre  de los Programas', 'Modalidad', 'Precio', 'carga_horaria',
       'precio_por_hora'],
      dtype='object')

In [69]:
data= df_dim_curso_perfil.groupby(['curso_perfil','modalidad']).count().reset_index()
data = data[['curso_perfil','modalidad']]
data

,curso_perfil,modalidad
0,3D STUDIO MAX,PRESENCIAL
1,3DS MAX + VRAY - VISUALIZACIÓN ARQUITECTÓNICA,ONLINE
2,3DS MAX +VRAY -VISUALIZACIÓN ARQUITECTÓNICA,PRESENCIAL
3,ADMINISTRACIÓN DE LA PLATAFORMA MOODLE,ONLINE
4,ADMINISTRACIÓN DE LA PLATAFORMA MOODLE,PRESENCIAL
...,...,...
351,WATERCAD & SEWERCAD -DISEÑO DE REDES DE AGUA P...,PRESENCIAL
352,WORD AVANZADO,ONLINE
353,WORD BASICO,ONLINE
354,WORD INTERMEDIO,ONLINE


In [76]:
arr = []

for i in range(len(df_coec['Nombre  de los Programas'])):
    # Similaridad 
    sub_arr = []
    for j in range(len(data['curso_perfil'])):
        curso_fuente = df_coec['Nombre  de los Programas'].iloc[i]
        curso_comparar = data['curso_perfil'].iloc[j]
        similary = lev.similarity(curso_fuente,curso_comparar)
        aux_similary=0
        if curso_comparar in curso_fuente :
            similary = 1
        else:
            arr_curso = curso_comparar.split(' ')
            for word in arr_curso:
                if word in curso_comparar:
                    aux_similary+=1/len(arr_curso)
        if aux_similary > similary:
            similary = aux_similary
        sub_arr.append(similary)
    arr.append(sub_arr)


In [77]:
res = pd.DataFrame(data=arr,index=df_coec['Nombre  de los Programas'],columns=data['curso_perfil'])
res

curso_perfil,3D STUDIO MAX,3DS MAX + VRAY - VISUALIZACIÓN ARQUITECTÓNICA,3DS MAX +VRAY -VISUALIZACIÓN ARQUITECTÓNICA,ADMINISTRACIÓN DE LA PLATAFORMA MOODLE,ADMINISTRACIÓN DE LA PLATAFORMA MOODLE,ADMINISTRACIÓN DE LA PLATAFORMA MOODLE,ADMINISTRACIÓN DE LINUX AVANZADA,ADMINISTRACIÓN DE LINUX BÁSICA,ADMINISTRACIÓN DE SERVICIOS DE RED EN LINUX,ADMINISTRACIÓN LINUX,...,VIRTUALIZACIÓN DE SERVIDORES EN LINUX,VISIÓN ARTIFICIAL CON RASPBERRY PI,VRAY PARA SKETCHUP - VISUALIZACIÓN ARQUITECTÓNICA,VRAY PARASKETCHUP - VISUALIZACIÓN ARQUITECTÓNICA,WATERCAD & SEWERCAD - DISEÑO DE REDES DE AGUA POTABLE Y ALCANTARILLADO SANITARIO,WATERCAD & SEWERCAD -DISEÑO DE REDES DE AGUA POTABLE Y ALCANTARILLADO SANITARIO,WORD AVANZADO,WORD BASICO,WORD INTERMEDIO,WORDPRESS
Nombre de los Programas,,,,,,,,,,,,,,,,,,,,,
programming essentials in python,0.062500,0.044444,0.046512,0.052632,0.052632,0.052632,0.000000,0.031250,0.069767,0.031250,...,0.054054,0.000000,0.040816,0.041667,0.037500,0.037975,0.031250,0.031250,0.031250,0
ccna cybersecurity operations,0.068966,0.044444,0.046512,0.026316,0.026316,0.026316,0.000000,0.000000,0.046512,0.034483,...,0.000000,0.058824,0.040816,0.041667,0.025000,0.025316,0.034483,0.034483,0.034483,0
ccna1 v7: introducción a redes,0.066667,0.066667,0.069767,0.052632,0.052632,0.052632,0.031250,0.000000,0.069767,0.033333,...,0.081081,0.029412,0.061224,0.041667,0.050000,0.050633,0.033333,0.033333,0.033333,0
"ccna3 v7: redes empresariales, seguridad y automatización",0.052632,0.070175,0.070175,0.052632,0.052632,0.052632,0.052632,0.052632,0.052632,0.017544,...,0.052632,0.052632,0.052632,0.035088,0.062500,0.063291,0.017544,0.017544,0.017544,0
fundamentos de ciberseguridad,0.034483,0.044444,0.046512,0.052632,0.052632,0.052632,0.062500,0.033333,0.046512,0.034483,...,0.054054,0.029412,0.040816,0.041667,0.025000,0.025316,0.034483,0.034483,0.034483,0
iot fudamentals: connecting things,0.058824,0.066667,0.046512,0.052632,0.052632,0.052632,0.029412,0.058824,0.046512,0.029412,...,0.054054,0.000000,0.061224,0.062500,0.037500,0.037975,0.029412,0.029412,0.029412,0
social media data analytics,0.074074,0.066667,0.069767,0.052632,0.052632,0.052632,0.031250,0.033333,0.046512,0.037037,...,0.027027,0.058824,0.061224,0.041667,0.037500,0.037975,0.037037,0.037037,0.037037,0
"growth hacking: estrategias, técnicas y acciones para generar tráfico a tu sitio web y redes sociales (seo, sem, analytics).",0.016129,0.048387,0.032258,0.032258,0.032258,0.032258,0.024194,0.024194,0.048387,0.008065,...,0.032258,0.032258,0.040323,0.032258,0.096774,0.088710,0.008065,0.008065,0.008065,0
facebook e instagram para emprendedores,0.051282,0.066667,0.046512,0.025641,0.025641,0.025641,0.025641,0.051282,0.023256,0.025641,...,0.000000,0.076923,0.061224,0.041667,0.050000,0.050633,0.025641,0.025641,0.025641,0


In [79]:
res.max().max()

0.13793103448275867

In [74]:
res['3D STUDIO MAX'][res['3D STUDIO MAX']==res['3D STUDIO MAX'].max()]

Nombre  de los Programas
social media data analytics    0.074074
Name: 3D STUDIO MAX, dtype: float64

In [ ]:




# Realizar un procesado de texto en donde se contenga los nombres de los cursos por modalidad en caso de OC el resto de cursos solo se tomará un promedio general estimado de estos cursos

# Procesado de texto:
#   Aplicar a lista de las palabras clave de tecnologia
#   Validar si un curso es igual a la lista de costos de cursos
#   Buscar simetría en el nombre de los cursos en el sitio web y el nombre de los cursos de las listas de costos
#   Revisará los nombre de todos los cursos y determinará las palabras sin palabras_de_parada que se repitan más para obtener palabras clave en cuenta a cursos para buscarlas